Сначала импортируем все, что нужно:

In [1]:
import tensorflow as tf 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Чтобы не выполнять много раз однотипные операции, зададим функции для объявления полносвязного слоя и слоя нормализации по мини-батчам. Полносвязный слой задается уже давно знакомым нам образом:

In [2]:
def fullyconnected_layer(tensor, input_size, out_size):
    W = tf.Variable(tf.truncated_normal([input_size, out_size], stddev=0.1))
    b = tf.Variable(tf.truncated_normal([out_size], stddev=0.1))
    return tf.nn.tanh(tf.matmul(tensor, W) + b)

А для того, чтобы задать слой нормализации, сначала нужно вычислить соответствующие статистики и объявить переменные, а дальше, как это часто бывает, использовать готовую функцию из библиотеки TensorFlow:

In [3]:
def batchnorm_layer(tensor, size):
    batch_mean, batch_var = tf.nn.moments(tensor, [0])
    beta = tf.Variable(tf.zeros([size]))
    scale = tf.Variable(tf.ones([size]))
    return tf.nn.batch_normalization(tensor, batch_mean, batch_var, beta, scale, 0.001)

Для эксперимента нам будет достаточно очень простой модели: мы возьмем полносвязную нейронную сеть с размерами слоев 784,100,100 и 10. Первый слой — входы, куда мы подаем значения пикселов изображения, последний — выходной слой, по одному нейрону на каждый из возможных классов изображения (то есть рукописные цифры от 0 до 9). Между промежуточными слоями мы и вставим слой нормализации по мини-батчам.

In [12]:
y = tf.placeholder(tf.float32, [None, 10])
x = tf.placeholder(tf.float32, [None, 784])
h1 = fullyconnected_layer(x, 784, 100)
h1_bn = batchnorm_layer(h1, 100)
h2 = fullyconnected_layer(h1_bn, 100, 100)
y_logit = fullyconnected_layer(h2, 100, 10)

Для обучения осталось только задать функцию ошибки и метод оптимизации:

In [14]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=y_logit, labels=y)
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

Далее стандартные действия.  
Инициализация переменных и создание сессии:

In [15]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

Запуск сессии:

In [18]:
for i in range(100):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_op, feed_dict={x: batch_xs, y: batch_ys})